In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
import pprint
import csv
import glob
import os

In [2]:
diab_hosp_pd = pd.read_csv('Resources/hospitalization.csv') #TRACT_FIPS
diab_hosp_pd = diab_hosp_pd.rename(columns={"TRACT_FIPS": "FIPS"})
diab_adults_pd = pd.read_csv('Resources/inadults.csv') #Census_Tract_FIPS
diab_adults_pd = diab_adults_pd.rename(columns={"Census_Tract_FIPS": "FIPS"})
diab_income_pd = pd.read_csv('Resources/income.csv') #FIPS
diab_food_pd = pd.read_csv('Resources/foodenviro.csv') #FIPS


merged_diabetes_df = pd.merge(diab_hosp_pd, diab_adults_pd, on = "FIPS")
merged_diabetes_df = pd.merge(merged_diabetes_df, diab_income_pd, on = "FIPS")
merged_diabetes_df = pd.merge(merged_diabetes_df, diab_food_pd, on = "FIPS")
merged_diabetes_df = merged_diabetes_df.dropna()
merged_diabetes_df.to_csv('Resources/merged.csv')
merged_diabetes_df = merged_diabetes_df.set_index('TRACT_NAME')
merged_diabetes_df.head()

,OBJECTID_x,FIPS,COUNTY_NAME,DIABETES_ADJRATE,DIABETES_L95CI,DIABETES_U95CI,DIABETES_STATEADJRATE,DIABETES_SL95CI,DIABETES_SU95CI,DIABETES_DISPLAY,...,lapop20,lapop20share,lalowi20,lalowi20share,lakids20,lakids20share,laseniors20,laseniors20share,lahunv20,lahunv20share
TRACT_NAME,,,,,,,,,,,,,,,,,,,,,
"Census Tract 9790, Fremont County, Colorado",1,8043979000,FREMONT,716.97,569.52,864.42,1267.78,1263.64,1271.91,Lower Than State Average,...,171.072852,4.941446,48.534714,1.401927,25.982590,5.648389,42.215172,4.972341,0.000000,0.000000
"Census Tract 9516, Garfield County, Colorado",2,8045951600,GARFIELD,602.99,495.07,710.91,1267.78,1263.64,1271.91,Lower Than State Average,...,13.208274,0.337635,1.711024,0.043738,1.390345,0.136042,1.390345,0.458860,0.000000,0.000000
"Census Tract 28.03, Larimer County, Colorado",3,8069002803,LARIMER,0.00,0.00,0.00,1267.78,1263.64,1271.91,No Events or Data Suppressed,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
"Census Tract 9632, Yuma County, Colorado",4,8125963200,YUMA,1390.98,1268.46,1513.50,1267.78,1263.64,1271.91,Not Different Than State Average,...,490.027359,9.006200,152.549299,2.803700,98.808630,6.852193,100.362885,11.188727,3.623084,0.168281
"Census Tract 24.01, Larimer County, Colorado",5,8069002401,LARIMER,529.77,370.49,689.05,1267.78,1263.64,1271.91,Lower Than State Average,...,973.481362,79.990252,274.443700,22.550838,94.899298,75.316903,294.619772,83.698799,9.445169,1.645500


LI = low income LA = low access

Low-income and low-access tract measured at 1 mile and 10 miles Definition: A low-income tract with at least 500 people, or 33 percent of the population, living more than 1 mile (urban areas) or more than 10 miles (rural areas) from the nearest supermarket, supercenter, or large grocery store.

Low-income and low-access tract measured at 1 and 20 miles Definition: A low-income tract with at least 500 people, or 33 percent of the population, living more than 1 mile (urban areas) or more than 20 miles (rural areas) from the nearest supermarket, supercenter, or large grocery store.

Low-income and low-access tract using vehicle access and at 20 miles Definition: A low-income tract in which at least one of the following is true: at least 100 households are located more than ½ mile from the nearest supermarket and have no vehicle access; or at least 500 people, or 33 percent of the population, live more than 20 miles from the nearest supermarket, regardless of vehicle availability.

In [5]:
narrowed_diab_df = merged_diabetes_df[['COUNTY_NAME', 'DIABETES_ADJRATE', 'DIABETES_STATEADJRATE',
                    'Adult_Population_Age18_and_over', 'Diabetes_Census_Tract_Estimate', 
                    'Population_Total', 'Population_Density_PerLandSquareMile',
                    'Percent_Poverty_AllPeople_Income_Below_Pov_Level', 'Poverty_Median_Household_Income', 
                    'Poverty_Mean_Household_Income',
                    'Poverty_Per_Capita_Income', 'LILATracts_1And10', 'LILATracts_halfAnd10', 'LILATracts_1And20', 
                                       'LILATracts_Vehicle', 'Urban', 'Rural', 'LowIncomeTracts']]

narrowed_diab_df = narrowed_diab_df.rename(columns={'COUNTY_NAME':'County', 'DIABETES_ADJRATE':'Hospitalization_Census_Tract_Rate_per_100,000',
                                                    'DIABETES_STATEADJRATE':'Hospitlization_State_Rate_per_100,000',
                                                    'Adult_Population_Age18_and_over':'Adult_Pop_Over_18', 
                                                    'Diabetes_Census_Tract_Estimate':'Census_Tract_Estimate', 
                                                    'Population_Total':'Pop_Total', 
                                                    'Population_Density_PerLandSquareMile':'Pop_Density/Sq_Mile',
                                                    'Percent_Poverty_AllPeople_Income_Below_Pov_Level': 'Percent_Poverty', 
                                                    'Poverty_Median_Household_Income': 'Median_Household_Income',
                                                    'Poverty_Per_Capita_Income': 'Per_Capita_Income', 
                                                    'Urban': 'Urban', 'Rural':'Rural', 'LowIncomeTracts': 'Low_Income_Tract'})

ct_total_count = narrowed_diab_df['County'].count()
narrowed_diab_df.to_csv('Resources/narrowed.csv')
narrowed_diab_df.head()

,County,"Hospitalization_Census_Tract_Rate_per_100,000","Hospitlization_State_Rate_per_100,000",Adult_Pop_Over_18,Census_Tract_Estimate,Pop_Total,Pop_Density/Sq_Mile,Percent_Poverty,Median_Household_Income,Poverty_Mean_Household_Income,Per_Capita_Income,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,Urban,Rural,Low_Income_Tract
TRACT_NAME,,,,,,,,,,,,,,,,,,
"Census Tract 9790, Fremont County, Colorado",FREMONT,716.97,1267.78,2994,10.0,3471,3.4,9.7,49483.0,61677.0,29749,1,1,0,0,0,1,1
"Census Tract 9516, Garfield County, Colorado",GARFIELD,602.99,1267.78,2800,5.0,3656,6.4,14.5,59154.0,82512.0,32362,0,0,0,0,0,1,0
"Census Tract 28.03, Larimer County, Colorado",LARIMER,0.00,1267.78,97,1.6,97,0.4,38.1,43882.0,41385.0,27278,0,0,0,0,0,1,0
"Census Tract 9632, Yuma County, Colorado",YUMA,1390.98,1267.78,4381,8.0,5467,5.3,13.4,44315.0,53229.0,24196,1,1,0,0,0,1,1
"Census Tract 24.01, Larimer County, Colorado",LARIMER,529.77,1267.78,1190,8.0,1310,3.3,10.2,61111.0,83656.0,39123,1,1,1,1,0,1,1
